Demo Pyspark: WordCount

Tool: Google Colab

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 65kB/s 
     |████████████████████████████████| 204kB 34.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=619f1f51a9c31cfab7d1d056e4a032f2345f45ef36cc352bc2ac0b40439a3c5a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 13 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [16]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [17]:
downloaded = drive.CreateFile({'id': '1xsjak6GE0LyB3kW3v8DkocsUhGsaQ4Pg'})
downloaded.GetContentFile('demo.txt')

In [20]:
from pyspark import SparkContext
import pandas as pd

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [21]:
input = sc.textFile('demo.txt')

In [28]:
input.take(5)

['Video provides a powerful way to help you prove your point. When you click Online Video, you can paste in the embed code for the video you want to add. You can also type a keyword to search online for the video that best fits your document. To make your document look professionally produced, Word provides header, footer, cover page, and text box designs that complement each other. For example, you can add a matching cover page, header, and sidebar. Click Insert and then choose the elements you want from the different galleries. Themes and styles also help keep your document coordinated.',
 'When you click Design and choose a new Theme, the pictures, charts, and SmartArt graphics change to match your new theme. When you apply styles, your headings change to match the new theme. Save time in Word with new buttons that show up where you need them. To change the way a picture fits in your document, click it and a button for layout options appears next to it. When you work on a table, cli

In [29]:
WordCount = input.flatMap(lambda x: x.split())
WordCount.take(5)

['Video', 'provides', 'a', 'powerful', 'way']

In [30]:
WordCount = WordCount.map(lambda x: (x, 1))
WordCount = WordCount.reduceByKey(lambda x,y : x + y)
WordCount.take(5)

[('Video', 2), ('provides', 4), ('way', 3), ('help', 3), ('When', 5)]

In [31]:
WordCount_DF = WordCount.toDF(("Word", "Amount"))
WordCount_DF.orderBy(col("Amount").desc()).show()

+--------+------+
|    Word|Amount|
+--------+------+
|     you|    21|
|     the|    18|
|       a|    12|
|     and|    12|
|      to|    11|
|    your|    10|
|     can|     7|
|   click|     6|
|    that|     5|
|    When|     5|
|    want|     5|
|      in|     5|
|     new|     5|
|     for|     5|
|document|     4|
|   cover|     4|
|    Word|     4|
|   video|     4|
|provides|     4|
|   page,|     4|
+--------+------+
only showing top 20 rows

